## Left array shift

Given the array a of n numbers perform left shift on k positions where k <= n.
Possible approaches:
* Use temporary array (increases space complexity)
* Do everything in place in one run
    * Using GCD
    * Using my own algorithm (tricky rotations)

In [73]:
def get_array(length):
    return list(range(1, length + 1))

### Using temporary array

In [74]:
def array_left_rotation_temp_array(a, n, k):
    # don't need to shift if n == k
    if (n == k):
        return a
    temp_array = [None] * n
    i = 0
    while i < n:
        temp_array[(i - k + n) % n] = a[i]
        i += 1
    return temp_array

### GCD

In [75]:
def gcd(a, b):
    if (b == 0):
        return a
    return gcd(b, a % b)

In [76]:
def array_left_rotation_gcd(a, n, k):
    # don't need to shift if n == k
    if (n == k):
        return a
    
    step = gcd(n, k)
    i = 0
    while i < step:
        temp = a[i]
        j = i
        while True:
            nextInd = j + k
            if (nextInd > n -1):
                nextInd -= n
            if (nextInd ==i ):
                break
            a[j] = a[nextInd]
            j = nextInd
        #print(a)
        a[j] = temp
        i += 1
    
    return a
    

### Tricky rotations

Idea: move elements in the iterations while making only one single pass on the array. 

Each iteration we are moving elements within two adjucent blocks of the fixed iteration length. To make sure we can have such iterations we always want iteration length to be less or equal then n/2. To achieve this we do the following:
* if k <= n/2 we start from 0 and iteration length of k, otherwise we start from the end of the array with iteration length n - k
* Swap elements within two blocks of length k by going through elements of the first block one by one.
* At the end of the iteration start next iteration from the element that follows [preceeds] the end of the second block. 
* If iteration cannot be completed because second block goes out of boundaries we need to reduce iteration length by the number of steps we were able to make on current iteration. 
* Continue until we reach the end [beginning] of the array or reduce iteration length down to 0



In [77]:
def array_left_rotation_creative(a, n, k):
    # don't need to shift if n == k
    if (n == k):
        return a;
    
    if (2*(n - k) < n):
        iterLength = n - k
        start = n - 1
        backwards = True
    else:
        iterLength = k
        start = 0
        backwards = False
        
    flag = True
    iter = 0
    while flag:
        i = 0;
        while i < iterLength:
            iter = iter  + 1
            if (backwards):
                nextIndex = start - i - iterLength
                currIndex = start - i
            else:
                nextIndex = start + i + iterLength
                currIndex = start + i
            if nextIndex > n - 1 or nextIndex < 0:
                iterLength = iterLength - i
                break
            a[nextIndex], a[currIndex] = a[currIndex], a[nextIndex]
            i = i + 1
            # print(a)
        if (backwards):
            start = start - i
            condition = start - iterLength < 0 or iterLength == 0
        else:
            start = start + i
            condition = start + iterLength > (n - 1) or iterLength == 0
        if (condition):
            flag = False;
        
        if (iter > 999):
            raise Exception('Something wrong with iterations')
            break

    return a

## Tests

### Definition

In [78]:
import unittest as ut

def run_test_case(testFn):

    class TestLeftRotation(ut.TestCase):
        def test1(self):
            answer = testFn(get_array(7), 7, 1)
            self.assertSequenceEqual(answer, [2, 3, 4, 5, 6, 7, 1])
        def test2(self):
            answer = testFn(get_array(7), 7, 2)
            self.assertSequenceEqual(answer, [3, 4, 5, 6, 7, 1, 2])
        def test3(self):
            answer = testFn(get_array(7), 7, 3)
            self.assertSequenceEqual(answer, [4, 5, 6, 7, 1, 2, 3])
        def test4(self):
            answer = testFn(get_array(7), 7, 4)
            self.assertSequenceEqual(answer, [5, 6, 7, 1, 2, 3, 4])
        def test5(self):
            answer = testFn(get_array(7), 7, 5)
            self.assertSequenceEqual(answer, [6, 7, 1, 2, 3, 4, 5])
        def test6(self):
            answer = testFn(get_array(7), 7, 6)
            self.assertSequenceEqual(answer, [7, 1, 2, 3, 4, 5, 6])

        def test10_1(self):
            answer = testFn(get_array(10), 10, 1)
            self.assertSequenceEqual(answer, [2, 3, 4, 5, 6, 7, 8, 9, 10, 1])

        def test10_2(self):
            answer = testFn(get_array(10), 10, 2)
            self.assertSequenceEqual(answer, [3, 4, 5, 6, 7, 8, 9, 10, 1, 2])

        def test10_3(self):
            answer = testFn(get_array(10), 10, 3)
            self.assertSequenceEqual(answer, [4, 5, 6, 7, 8, 9, 10, 1, 2, 3])

        def test10_4(self):
            answer = testFn(get_array(10), 10, 4)
            self.assertSequenceEqual(answer, [5, 6, 7, 8, 9, 10, 1, 2, 3, 4])

        def test10_5(self):
            answer = testFn(get_array(10), 10, 5)
            self.assertSequenceEqual(answer, [6, 7, 8, 9, 10, 1, 2, 3, 4, 5])

        def test10_6(self):
            answer = testFn(get_array(10), 10, 6)
            self.assertSequenceEqual(answer, [7, 8, 9, 10, 1, 2, 3, 4, 5, 6])



    loader = ut.TestLoader()
    suite = ut.TestSuite()
    runner = ut.TextTestRunner()

    suite.addTests(loader.loadTestsFromTestCase(TestLeftRotation))
    runner.run(suite)

### Testing inplace tricky rotations

In [79]:
run_test_case(array_left_rotation_creative)

............
----------------------------------------------------------------------
Ran 12 tests in 0.020s

OK


In [80]:
import timeit
a = get_array(1000);
print(timeit.timeit("array_left_rotation_creative(a, 1000, 300)", setup="from __main__ import array_left_rotation_creative, a", number=10000))

3.9156354300212115


### Testing temp array

In [81]:
run_test_case(array_left_rotation_temp_array)

............
----------------------------------------------------------------------
Ran 12 tests in 0.021s

OK


In [82]:
import timeit
a = get_array(1000);
print(timeit.timeit("array_left_rotation_temp_array(a, 1000, 300)", setup="from __main__ import array_left_rotation_temp_array, a", number=10000))

2.3773374110169243


### Testing GCD

In [83]:
run_test_case(array_left_rotation_gcd)

............
----------------------------------------------------------------------
Ran 12 tests in 0.022s

OK


In [84]:
import timeit
a = get_array(1000);
print(timeit.timeit("array_left_rotation_gcd(a, 1000, 300)", setup="from __main__ import array_left_rotation_gcd, a", number=10000))

2.160941448993981
